# Explore cycle share data


## Installation requirements

You must install the following packages to follow this notebook:

```bash
conda install -c conda-forge altair vega_datasets vega
conda install -c conda-forge ipyleaflet
pip install dataset
```

# Import data

The data was downloaded from the [Kaggle Datasets](https://www.kaggle.com/pronto/cycle-share-dataset) repository,
and was originally provided by Pronto, the company that operates the cycle share system at Seattle, as part of an open data initiative.

On the Kaggle repository you will find a detailed data description, reproduced here for the reader convenience.
## Context

The Pronto Cycle Share system consists of 500 bikes and 54 stations located in Seattle. Pronto provides open data on individual trips, stations, and daily weather.

## Content

There are 3 datasets that provide data on the stations, trips, and weather from 2014-2016.

1. Station dataset
    
    - station_id: station ID number
    - name: name of station
    - lat: station latitude
    - long: station longitude
    - install_date: date that station was placed in service
    - install_dockcount: number of docks at each station on the installation date
    - modification_date: date that station was modified, resulting in a change in location or dock count
    - current_dockcount: number of docks at each station on 8/31/2016
    - decommission_date: date that station was placed out of service


2. Trip dataset
    
    - trip_id: numeric ID of bike trip taken
    - starttime: day and time trip started, in PST
    - stoptime: day and time trip ended, in PST
    - bike_id: ID attached to each bike
    - tripduration: time of trip in seconds
    - from_station_name: name of station where trip originated
    - to_station_name: name of station where trip terminated
    - from_station_id: ID of station where trip originated
    - to_station_id: ID of station where trip terminated
    - usertype: "Short-Term Pass Holder" is a rider who purchased a 24-Hour or 3-Day Pass; "Member" is a rider who purchased a Monthly or an Annual Membership
    - gender: gender of rider
    - birthyear: birth year of rider


3. Weather dataset contains daily weather information in the service area

In [1]:
# import the csv data in pandas 
import os
import pandas as pd
import numpy as np

data_dir = "cycle-share-dataset"

station = pd.read_csv(
    os.path.join(data_dir, "station.csv"), 
    parse_dates=["install_date","modification_date","decommission_date"]
)

trip = pd.read_csv(
    os.path.join(data_dir, "trip.csv"), 
    parse_dates=["starttime","stoptime"],
    skiprows=range(1,50794) # the first 50794 are duplicates ...
)
assert trip["trip_id"].nunique()==trip.shape[0]

weather = pd.read_csv(
    os.path.join(data_dir, "weather.csv"), 
    parse_dates=["Date"]
)

# Clean data

Before using any data you should check it thoroughly ! Let's check for instance the `station` dataset. The data description already provides a lot of information:

- the meaning (and therefore expected data-type) of each column
- there are 54 stations, uniquely identified by `station_id`

First let's see how many rows we have and list the columns along with their data-type.

In [2]:
print(f"station: {station.shape[0]} rows {station.shape[1]} columns\n")
print(station.dtypes)

station: 58 rows 9 columns

station_id                   object
name                         object
lat                         float64
long                        float64
install_date         datetime64[ns]
install_dockcount             int64
modification_date    datetime64[ns]
current_dockcount             int64
decommission_date    datetime64[ns]
dtype: object


We have 58 records and not 54. Are there any duplicated rows ? Let's how many distinct `station_id` we have:

In [3]:
n_station_id = station["station_id"].nunique()
print(f"There are {n_station_id} distinct station_id")

There are 58 distinct station_id


Ok, so we have no duplicate and exactly one record per `station_id`. There are in fact 58 stations in our dataset, not 54. 

Now let's take a look at the first rows:

In [4]:
station.head(5)

,station_id,name,lat,long,install_date,install_dockcount,modification_date,current_dockcount,decommission_date
0,BT-01,3rd Ave & Broad St,47.618418,-122.350964,2014-10-13,18,NaT,18,NaT
1,BT-03,2nd Ave & Vine St,47.615829,-122.348564,2014-10-13,16,NaT,16,NaT
2,BT-04,6th Ave & Blanchard St,47.616094,-122.341102,2014-10-13,16,NaT,16,NaT
3,BT-05,2nd Ave & Blanchard St,47.613110,-122.344208,2014-10-13,14,NaT,14,NaT
4,CBD-03,7th Ave & Union St,47.610731,-122.332447,2014-10-13,20,NaT,20,NaT


Note that all fields are consistent with the description: `lat` and `long` look like latitutde and longitude, `name` as a station name, the counts look like counts and dates like dates.

However the `modification_date` and `decommission_date` are all missing in the first 5 rows: values are all `NaT` meaning Not-a-Time. For the dates let's see how many missing values we have, and also the min and max date: 

In [5]:
for date_column in ["install_date", "modification_date", "decommission_date"]:
    n_missing = station[date_column].isnull().sum()
    date_min = station[date_column].min()
    date_max = station[date_column].max()
    print(f"{date_column}: {n_missing} are missing min={date_min} max={date_max}")

install_date: 0 are missing min=2014-10-13 00:00:00 max=2016-08-09 00:00:00
modification_date: 41 are missing min=2015-02-20 00:00:00 max=2016-08-09 00:00:00
decommission_date: 54 are missing min=2015-10-29 00:00:00 max=2016-08-09 00:00:00


That makes sense: all have an installation date, a few have been modified and only 58 - 54 = 4 were decommissioned.
Probably the 54 in the data description was referring to stations still in service.

We can list here the 4 out of service stations:

In [6]:
station[station.decommission_date.notnull()]

,station_id,name,lat,long,install_date,install_dockcount,modification_date,current_dockcount,decommission_date
10,CD-01,12th Ave & E Yesler Way,47.602103,-122.316923,2015-05-22,16,2016-08-09,0,2016-08-09
26,FH-01,Frye Art Museum / Terry Ave & Columbia St,47.607281,-122.324783,2014-10-13,16,2016-03-18,0,2016-03-18
38,SLU-18,Dexter Ave & Denny Way,47.618285,-122.342205,2014-10-13,20,2015-11-02,0,2016-07-02
46,UW-01,UW McCarty Hall / Whitman Ct,47.660268,-122.304826,2014-10-13,16,2015-10-29,0,2015-10-29


What about the remaining numeric columns `lat`, `long`, `install_dockcount` and `current_dockcount`, does the range of values make sense ? Are their any missing values ? Let's use the `describe()` method to get a quick statistical summary for each column, and `.T` to transpose the summary stats dataframe.

In [7]:
station.describe().T

,count,mean,std,min,25%,50%,75%,max
lat,58.0,47.624796,0.019066,47.598488,47.613239,47.618591,47.627712,47.666145
long,58.0,-122.327242,0.014957,-122.355230,-122.338735,-122.328207,-122.316691,-122.284119
install_dockcount,58.0,17.586207,3.060985,12.000000,16.000000,18.000000,18.000000,30.000000
current_dockcount,58.0,16.517241,5.117021,0.000000,16.000000,18.000000,18.000000,26.000000


They are no missing values (**count** counts the number of non-null values), and every range of values makes sense. For instance, Seattle is located at $47°36′N-122°19′W$.

Now let's check the `trip` dataset. If the start and stop time are consitent we expect that 

- `stoptime` $>$ `startime`,
- `tripduration` $\simeq$ `stoptime` $-$ `startime` (in seconds).

But actually there are a few errors !

In [8]:
print(f"trip: {trip.shape[0]} records")
# count number of trips where stoptime < starttime
n_time_travel = (trip["stoptime"] < trip["starttime"]).sum()
print(f"The {n_time_travel} trips for which stoptime < startime:")
# show the few outliers
trip.query("stoptime < starttime")

trip: 236065 records
The 7 trips for which stoptime < startime:


,trip_id,starttime,stoptime,bike_id,tripduration,from_station_name,to_station_name,from_station_id,to_station_id,usertype,gender,birthyear
7040,8660,2014-11-02 01:29:00,2014-11-02 01:12:00,SEA00384,2571.017,Pine St & 9th Ave,Westlake Ave & 6th Ave,SLU-16,SLU-15,Short-Term Pass Holder,NaN,NaN
7043,8663,2014-11-02 01:29:00,2014-11-02 01:11:00,SEA00205,2513.548,Pine St & 9th Ave,Westlake Ave & 6th Ave,SLU-16,SLU-15,Short-Term Pass Holder,NaN,NaN
7044,8666,2014-11-02 01:31:00,2014-11-02 01:11:00,SEA00430,2398.734,Pine St & 9th Ave,Westlake Ave & 6th Ave,SLU-16,SLU-15,Short-Term Pass Holder,NaN,NaN
7045,8667,2014-11-02 01:37:00,2014-11-02 01:12:00,SEA00112,2074.203,Pine St & 9th Ave,Westlake Ave & 6th Ave,SLU-16,SLU-15,Short-Term Pass Holder,NaN,NaN
7046,8669,2014-11-02 01:44:00,2014-11-02 01:04:00,SEA00247,1201.414,2nd Ave & Vine St,Key Arena / 1st Ave N & Harrison St,BT-03,SLU-19,Member,Male,1989.0
7047,8670,2014-11-02 01:52:00,2014-11-02 01:07:00,SEA00460,918.994,2nd Ave & Vine St,Summit Ave & E Denny Way,BT-03,CH-01,Member,Male,1985.0
7048,8672,2014-11-02 01:59:00,2014-11-02 01:25:00,SEA00481,1565.225,UW Magnuson Health Sciences Center Rotunda / C...,Children's Hospital / Sandpoint Way NE & 40th ...,UW-10,DPD-03,Short-Term Pass Holder,NaN,NaN


In [9]:
# let's recompute the trip duration in seconds from stop and start time:
trip["computed_duration"] = (trip["stoptime"]-trip["starttime"]).dt.seconds
# and see if there is more than a 1min = 60s difference with tripduration:
trip["over_1min"] = (trip["computed_duration"]-trip["tripduration"]).abs() > 60
# show the few outliers
print("Trips with over 1min difference between stoptime-starttime and tripduration")
# we focus on the stoptime > startime trips, the stoptime < startime trips are shown just above
trip.query("(stoptime > starttime) & over_1min")

Trips with over 1min difference between stoptime-starttime and tripduration


,trip_id,starttime,stoptime,bike_id,tripduration,from_station_name,to_station_name,from_station_id,to_station_id,usertype,gender,birthyear,computed_duration,over_1min
7041,8661,2014-11-02 01:29:00,2014-11-02 01:35:00,SEA00371,3986.146,Cal Anderson Park / 11th Ave & Pine St,Cal Anderson Park / 11th Ave & Pine St,CH-08,CH-08,Short-Term Pass Holder,NaN,NaN,360,True
7042,8662,2014-11-02 01:29:00,2014-11-02 01:35:00,SEA00170,3978.278,Cal Anderson Park / 11th Ave & Pine St,Cal Anderson Park / 11th Ave & Pine St,CH-08,CH-08,Short-Term Pass Holder,NaN,NaN,360,True


There are actually very few errors (9 out of 236065). Let's filter out the bad rows, and drop the utility columns
`computed_duration` and `over_1min` we have created for the sanity check.

In [10]:
# tilde is the logical NOT operator
trip = trip.query("(stoptime > starttime) & ~over_1min")
# dropping columns
trip = trip.drop(
    columns=["computed_duration", "over_1min"]
)
print(f"Filtered trip dataset: we now have {trip.shape[0]} records")

Filtered trip dataset: we now have 236056 records


Let's do one final check. You may have notice that the trip dataset contains the ID of the stations where the trip originated and terminated. But do we recover every `station_id` of the `trip` dataset in the `station` dataset ?

In [11]:
station_ids = station["station_id"].unique().tolist()
from_station_ids = trip["from_station_id"].unique().tolist()
to_station_ids = trip["to_station_id"].unique().tolist()
trip_station_ids = set(from_station_ids + to_station_ids)
not_in_station = [
    station_id for station_id in trip_station_ids
    if station_id not in station_ids
]
print(
    f"{len(not_in_station)} / {len(trip_station_ids)} ids not recovered in station", 
    not_in_station
)

3 / 61 ids not recovered in station ['Pronto shop 2', 'Pronto shop', '8D OPS 02']


These station ids indeed seem special, maybe they correspond to a repair / maintetance shop ?

## Summary

This dataset is very clean : all fields have a clear meaning that match the data type, no weird values, each row corresponds to exactly one station. Note that such high quality data is more the exception than the rule...

Usually data is very messy, and you will spend a considerable amount of time cleaning it.
Unfortunately people are relentlessly creative to mess things up: often each dataset is messy in its own unique way.
Fields can be incoherent (start time $>$ end time), using $-1$ or $999$ as missing values, aberrant values, duplicated rows, unintelligble column names, etc.

## Exercise

Your turn ! Check that:

- each trip corresponds to a single record in the `trip` dataset (no duplicates)
- the values for `gender`, `usertype` and `birthyear` make sense
- the number of bikes agrees with the data description
- the values in `weather` data make sense

From wikipedia:
- The dew point is the temperature to which air must be cooled to become saturated with water vapor.
- A gust or wind gust is a brief increase in the speed of the wind, usually less than 20 seconds.

In [14]:
# %load exo1.py

# Basic visualization

# Reshape data

# Advanced visualization